### Import libraries

In [156]:
import nltk
import gensim
import numpy as np
import json
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

In [157]:
#import documents
main_doc = 'intents.json'
text_doc = 'test.txt'

In [158]:
with open(main_doc,) as f:
    data = json.load(f)
    for Dept in data['Economics']:
        course = Dept['Course']
        question = Dept['question']

In [159]:
file_docs = []

#load document and make sentence tokens

with open(main_doc,) as f:
    data = json.load(f)
    for Dept in data['Economics']:
        course = Dept['Course']
        question = Dept['question']
        for answers in Dept['answers']:
            tokens = sent_tokenize(answers)
            for line in tokens:
                file_docs.append(line)
        print("Number of sentences:",len(file_docs))
        #make sentence into word tokens
        gen_docs = [[w.lower() for w in word_tokenize(text)] 
                    for text in file_docs]
        print(gen_docs, "\n")
#Check how tokenized words occur in the sentence and make weighted summary. 
#Words will least occurence get high weights, while words with high occurence get low weights or disappear altogether
tf_idf = gensim.models.TfidfModel(corpus)
for doc in tf_idf[corpus]:
    print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])

Number of sentences: 2
[['the', 'branch', 'of', 'economics', 'concerned', 'with', 'the', 'use', 'of', 'mathematical', 'methods', '(', 'especially', 'statistics', ')', 'in', 'describing', 'economic', 'systems', '.'], ['econometrics', 'sutudies', 'how', 'mathematics', 'relates', 'with', 'economics']] 

[['(', 0.21], [')', 0.21], ['.', 0.21], ['branch', 0.21], ['concerned', 0.21], ['describing', 0.21], ['economic', 0.21], ['especially', 0.21], ['in', 0.21], ['mathematical', 0.21], ['methods', 0.21], ['of', 0.43], ['statistics', 0.21], ['systems', 0.21], ['the', 0.43], ['use', 0.21]]
[['econometrics', 0.45], ['how', 0.45], ['mathematics', 0.45], ['relates', 0.45], ['sutudies', 0.45]]


In [160]:
#make dictionary mapping by converting each word to unique ids.
dictionary = Dictionary(gen_docs)
print(dictionary.token2id)
#ceate bag of words
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

{'(': 0, ')': 1, '.': 2, 'branch': 3, 'concerned': 4, 'describing': 5, 'economic': 6, 'economics': 7, 'especially': 8, 'in': 9, 'mathematical': 10, 'methods': 11, 'of': 12, 'statistics': 13, 'systems': 14, 'the': 15, 'use': 16, 'with': 17, 'econometrics': 18, 'how': 19, 'mathematics': 20, 'relates': 21, 'sutudies': 22}


In [161]:
file2_docs = []
with open (text_doc) as f:
    tokens = sent_tokenize(f.read())
    for line in tokens:
        file2_docs.append(line)

print("Number of sentences:",len(file2_docs))  
for line in file2_docs:
    query_doc = [w.lower() for w in word_tokenize(line)]
    query_doc_bow = dictionary.doc2bow(query_doc)


Number of sentences: 1


Term frequency is how often the word shows up in the document and inverse document frequency scales the value by how rare the word is in the corpus. In simple terms, words that occur more frequently across the documents get smaller weights.

### Creating similarity measure object

In [162]:
 # create index for main doc corpus and store matrix in wokrdir
sims = gensim.similarities.Similarity('workdir/',tf_idf[corpus],
                                        num_features=len(dictionary))

#### Document similarities to query

In [163]:
# perform a similarity query against the corpus
query_doc_tf_idf = tf_idf[query_doc_bow]
# print(document_number, document_similarity)
print('Comparing Result:', sims[query_doc_tf_idf]) 

Comparing Result: [0.88040876 0.10259783]


In [164]:
sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
print(sum_of_sims)

0.9830066


In [165]:
percentage_of_similarity = round(float((sum_of_sims / len(file_docs)) * 100))
print(f'Average similarity float: {float(sum_of_sims / len(file_docs))}')
print(f'Average similarity percentage: {float(sum_of_sims / len(file_docs)) * 100}')
print(f'Average similarity rounded percentage: {percentage_of_similarity}')

Average similarity float: 0.4915032982826233
Average similarity percentage: 49.15032982826233
Average similarity rounded percentage: 49


In [166]:
avg_sims = [] # array of averages

# calculate average of similarity for each query doc
avg = sum_of_sims / len(file_docs)
# print average of similarity for each query doc
print(f'avg: {sum_of_sims / len(file_docs)}')
# add average values into array
avg_sims.append(avg)  
# calculate total average
total_avg = ((np.sum(avg_sims, dtype=np.float)) / len(file2_docs))
# round the value and multiply by 100 to format it as percentage
percentage_of_similarity = round(float(total_avg) * 100)
# if percentage is greater than 100
# that means documents are almost same
if percentage_of_similarity >= 100:
    percentage_of_similarity = 100

avg: 0.4915032982826233


#### Prediction

In [167]:
percentage = "{:.0%}".format((sum_of_sims / len(file_docs)))
print(percentage)
value = sum_of_sims / len(file_docs) * 100
score = int(value)
print(score)

49%
49


In [177]:
def getgrade(score):
    if (score >= 90.0):
        return 'A'
    if(score >= 70.0 and score <= 80.0):
        return 'B'
    if(score >= 50.0 and score <= 60.0):
        return 'C'
    if(score >= 40.0):
        return 'D'
    if(score <= 30.0):
        return 'F'

In [178]:
def canquest(s):
    str = " "
    return (str.join(s))

In [180]:
print('Course title:',course,'\n')
print('Question:',question,'\n',)
print('Candidate answer:', canquest(file2_docs))
print('Correct answers:', canquest(file_docs))
print('Similarity to answer(percentage):', percentage)
print('Similarity to answer:', getgrade(score))

Course title: ecn314 

Question: ['What is econometrics'] 

Candidate answer: econometrics is the branch of economics concerned with the use of mathematical methods (especially statistics) using ecnomomics principles.
Correct answers: the branch of economics concerned with the use of mathematical methods (especially statistics) in describing economic systems. Econometrics sutudies how mathematics relates with economics
Similarity to answer(percentage): 49%
Similarity to answer: D
